In [130]:
import pandas as pd
import numpy as np
import os
from sqlalchemy import create_engine
from scipy import stats
import math as m

In [2]:
driver = 'mysql+pymysql:'
user = 'root'
password = '19693903'
ip = '127.0.0.1'
database = 'project_4'

In [3]:
connection_string = f'{driver}//{user}:{password}@{ip}/{database}'
engine = create_engine(connection_string)

In [12]:
query = """
SELECT *
FROM flights
"""
query2 = """
SELECT * FROM airports
"""

In [75]:
data= pd.read_sql(query, engine)
airports_info = pd.read_sql(query2, engine)
data = data.merge(airports_info, left_on = 'depart_airpt', right_on='iata_code')

In [76]:
data = data.drop(columns = ('iata_code'), axis = 1).rename(columns = {'country_code':'depart_country'})
data = data.astype({'depart_delay': int,'arri_delay':int})
data['Total_delay'] = data['depart_delay'] + data['arri_delay']

In [77]:
q1, q3= np.percentile(data['Total_delay'],[25,75])
iqr = q3 - q1
upper_bound = q3 +(3.5 * iqr)
upper_bound

149.5

In [78]:
data = data[data['Total_delay'] <= upper_bound]

## Checking delay status:
By law, if a flight's departure delay time is longer than 15 min, it should be classify as **delayed**

In [79]:
data['Delay_status'] = np.where(data.depart_delay >= 15, 1,0)

### Classiying datetime into morning, afternoon and night

In [113]:
def classify_day(x):
    if x.hour <=12.0:
        return 0
    elif (x.hour > 12.0) and (x.hour <= 18.0):
        return 1
    else:
        return 2

In [115]:
data['day_range'] = data['schd_depart_time'].apply(classify_day)

### Europe summary statistics

In [119]:
eu_delays = data[['depart_delay', 'arri_delay','Total_delay']]
eu_delays =  eu_delays[np.logical_and(eu_delays['depart_delay']>0, eu_delays['arri_delay']>0)]
eu_delays.describe()

,depart_delay,arri_delay,Total_delay
count,33228.000000,33228.000000,33228.000000
mean,21.507433,19.241062,40.748495
std,16.579282,16.739118,32.087055
min,1.000000,1.000000,2.000000
25%,9.000000,7.000000,17.000000
50%,17.000000,14.000000,31.000000
75%,30.000000,27.000000,56.000000
max,86.000000,133.000000,149.000000


In [128]:
eu_dep_delay_mean = eu_delays.depart_delay.mean()
eu_dep_delay_std = eu_delays.depart_delay.std()
eu_dep_delay_var = eu_delays.depart_delay.var()
eu_t_delay_mean = eu_delays.Total_delay.mean()
eu_t_delay_std = eu_delays.Total_delay.std()
eu_t_delay_var = eu_delays.Total_delay.var()

europe_delay_mean = 12.0
#https://www.eurocontrol.int/sites/default/files/2019-07/eurocontrol_top_stats_-_june_2019.pdf

### Spain summary statistics


In [121]:
data_es = data[data['depart_country'] == 'ES']
es_delays =  data_es[np.logical_and(data_es['depart_delay']>0, data_es['arri_delay']>0)]

In [134]:
es_delays.describe()

,depart_delay,arri_delay,Total_delay,Delay_status,day_range
count,6861.000000,6861.000000,6861.000000,6861.000000,6861.000000
mean,20.951902,18.884273,39.836175,0.532138,0.464947
std,17.761204,17.037834,33.396832,0.499002,0.553391
min,1.000000,1.000000,2.000000,0.000000,0.000000
25%,8.000000,6.000000,15.000000,0.000000,0.000000
50%,15.000000,13.000000,28.000000,1.000000,0.000000
75%,29.000000,26.000000,55.000000,1.000000,1.000000
max,86.000000,90.000000,149.000000,1.000000,2.000000


In [122]:
es_dep_delay_mean = es_delays.depart_delay.mean()
es_dep_delay_std = es_delays.depart_delay.std()
es_dep_delay_var = es_delays.depart_delay.var()
es_t_delay_mean = es_delays.Total_delay.mean()
es_tl_delay_std = es_delays.Total_delay.std()
es_t_delay_var = es_delays.Total_delay.var()

### Barcelona summary statistics


In [123]:
data_bcn = data[data['depart_airpt'] == 'BCN']
bnc_delays =  data_bcn[np.logical_and(data_bcn['depart_delay']>0, data_bcn['arri_delay']>0)]

In [139]:
bnc_delays.describe()

,depart_delay,arri_delay,Total_delay,Delay_status,day_range
count,1627.000000,1627.000000,1627.000000,1627.000000,1627.000000
mean,22.258144,19.788568,42.046712,0.578365,0.406269
std,16.891186,15.854296,31.190941,0.493973,0.540179
min,1.000000,1.000000,3.000000,0.000000,0.000000
25%,9.000000,7.000000,17.000000,0.000000,0.000000
50%,18.000000,16.000000,33.000000,1.000000,0.000000
75%,32.000000,28.000000,58.000000,1.000000,1.000000
max,86.000000,74.000000,147.000000,1.000000,2.000000


In [124]:
bnc_dep_delay_mean = bnc_delays.depart_delay.mean()
bnc_dep_delay_std = bnc_delays.depart_delay.std()
bnc_dep_delay_var = bnc_delays.depart_delay.var()
bnc_t_delay_mean = bnc_delays.Total_delay.mean()
bnc_t_delay_std = bnc_delays.Total_delay.std()
bnc_t_delay_var = bnc_delays.Total_delay.var()

### Madrid summary statistics


In [125]:
data_mad = data[data['depart_airpt'] == 'MAD']
mad_delays =  data_mad[np.logical_and(data_mad['depart_delay']>0, data_mad['arri_delay']>0)]

In [126]:
mad_dep_delay_mean = mad_delays.depart_delay.mean()
mad_dep_delay_std = mad_delays.depart_delay.std()
mad_dep_delay_var = mad_delays.depart_delay.var()
mad_t_delay_mean = mad_delays.Total_delay.mean()
mad_t_delay_std = mad_delays.Total_delay.std()
mad_t_delay_var = mad_delays.Total_delay.var()

# Hypothesis testing

### 1- Its is more likely to have delay time in Spain airports than in the rest of europe. 

H0: The mean departure time in minutes in Spain is equal or lower than in the rest of Europe


H1: The mean departure time in minutes in Spain is greater than in the rest of Europe

#### Testing with the european departure delay mean by Eurocontrol

In [138]:
#With a condifence level of 5%, lets test to see if we can reject our null hypotehsis.
alpha = 0.05

result_ttest = stats.ttest_1samp(es_delays['depart_delay'], europe_delay_mean)

if (result_ttest[0] > 0) & (result_ttest[1] < alpha):
    print (f"Reject null hypothesis, we have evidence for alternative hypothesis\n")
else:
    print('We dont have statistical evidence to reject the null hypothesis')

print(result_ttest)

Reject null hypothesis, we have evidence for alternative hypothesis

Ttest_1sampResult(statistic=41.748091665097334, pvalue=0.0)


#### Testing with the european departure delay mean of my sample

In [137]:
#With a condifence level of 5%, lets test to see if we can reject our null hypotehsis.
alpha = 0.05

result_ttest = stats.ttest_1samp(es_delays['depart_delay'], eu_dep_delay_mean)

if (result_ttest[0] > 0) & (result_ttest[1] < alpha):
    print (f"Reject null hypothesis, we have evidence for alternative hypothesis\n")
else:
    print('We dont have statistical evidence to reject the null hypothesis\n')

print(result_ttest)

We dont have statistical evidence to reject the null hypothesis

Ttest_1sampResult(statistic=-2.5907764760355323, pvalue=0.009596235629558636)


### 2-The Barcelona airport is the airport with less delays time in average of Spain. 

H0: The mean departure time in minutes in BCN is equal or greater than in the rest of Spain.


H1: The mean departure time in minutes in BCN is lower than in the rest of Spain.

In [140]:
#With a condifence level of 5%, lets test to see if we can reject our null hypotehsis.

alpha = 0.05

result_ttest = stats.ttest_1samp(es_delays['depart_delay'], es_dep_delay_mean)

if (result_ttest[0] < 0) & (result_ttest[1] < alpha):
    print (f"Reject null hypothesis, we have evidence for alternative hypothesis\n")
else:
    print('We dont have statistical evidence to reject the null hypothesis\n')

print(result_ttest)

We dont have statistical evidence to reject the null hypothesis

Ttest_1sampResult(statistic=0.0, pvalue=1.0)
